<h2>This bot trades 15EMA in Nifty by buying call options</h2>
<hr>

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib as ta
import time
from pushover import Pushover
import logging
from kiteconnect import KiteConnect

In [2]:
po = Pushover("an3he6x85bwsh97t87goxwu462yru6")
po.user("uuev1ffw9cepf5iquv3wu8hk29xo3m")

In [3]:
def myround(x):
    return 50 * round(x/50)

In [4]:
def Watch(target,sl,tsym,entry,n,d,en):
    while(True):
        data = yf.download(tickers='^NSEI', period='2d', interval='15m')
        data['5EMA'] = ta.EMA(data['Close'],5)
        print(data.tail())

        if(data['Low'].values[-2]-data['5EMA'].values[-2]>0.4):
            msg = po.msg("Watch the market")
            msg.set("title", "Possible Reversal")
            po.send(msg)

        if((data['Close'].values[-1]>target) or (data['High'].values[-1]>target)):
            msg = po.msg("Watch the market")
            msg.set("title", "Target Hit")
            po.send(msg)
            sl = en + 5
            en = target
            n += 1
            target = entry + (n*d)
            Watch(target,sl,tsym,entry,n,d,en)

        
        if(data['Close'].values[-1]<sl):
            try:
                s_order_id = kite.place_order(tradingsymbol=tsym,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_SELL,
                                quantity=50,
                                variety=kite.VARIETY_REGULAR,
                                order_type=kite.ORDER_TYPE_MARKET,
                                product=kite.PRODUCT_MIS)
                logging.info("Order placed. ID :{}".format(s_order_id))
                msg = po.msg("Trailing stop loss hit Check P&L ID :{}".format(s_order_id))
                msg.set("title", "Trailing stop Loss hit")
                po.send(msg)
            except Exception as e:
                msg = po.msg("Failed: "+str(e))
                logging.info(""+str(e))
                msg.set("title", "Trailing stop loss failed")
                po.send(msg)
            msg = po.msg("Square off position and Watch the market")
            msg.set("title", "Stop loss Hit")
            po.send(msg)
            return 
        
        time.sleep(5)

In [5]:
def Enter():
    while(True):
        data = yf.download(tickers='^NSEI', period='2d', interval='15m')
        data['5EMA'] = ta.EMA(data['Close'],5)
        print(data.tail())
        if(data['High'].values[-1] > data['5EMA'].values[-2] or data['Close'].values[-1]>data['5EMA'].values[-2]):
            sl = min(data['Low'].values[-3],data['Low'].values[-2],data['Low'].values[-4])
            tsym = "NIFTY23302"+str(myround(data['Close'][-1]))+"CE"
            try:
                b_order_id = kite.place_order(tradingsymbol=tsym,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=50,
                                variety=kite.VARIETY_REGULAR,
                                order_type=kite.ORDER_TYPE_MARKET,
                                product=kite.PRODUCT_MIS)
                logging.info("Order placed. ID :{}".format(b_order_id))
                msg = po.msg("Buy order placed ID :{}".format(b_order_id))
                msg.set("title", "CE bought")
                po.send(msg)
            except Exception as e:
                msg = po.msg("Failed: "+str(e))
                logging.info("Order placement failed: "+str(e))
                msg.set("title", "Buy Order Failed")
                po.send(msg)
            entry = data['Close'].values[-1]
            target = entry + (entry - sl)
            sle = ((entry-sl)*0.5)*50
            msg = po.msg("Expected max loss :{}".format(sle))
            msg.set("title", "Sl set")
            po.send(msg)
            n = 1
            d = entry - sl
            en = entry
            Watch(target,sl,tsym,entry,n,d,en)
            return 
        time.sleep(5)

In [6]:
def Trade():
    while(True):
        data = yf.download(tickers='^NSEI', period='2d', interval='15m')
        data['5EMA'] = ta.EMA(data['Close'],5)
        print(data.tail())
        if(data['High'].values[-2]<data['5EMA'].values[-2]):
            msg = po.msg("Watch market about to place order")
            msg.set("title", "Bullish signal 15EMA")
            po.send(msg)
            Enter()
            msg = po.msg("Rerun the bot")
            msg.set("title", "Trade cycle complete")
            po.send(msg)
            return 
        time.sleep(300)

In [8]:
Trade()

[*********************100%***********************]  1 of 1 completed
                             Open          High           Low         Close  \
Datetime                                                                      
2023-03-02 09:30:00  17375.750000  17393.550781  17364.150391  17388.150391   
2023-03-02 09:45:00  17388.750000  17393.449219  17348.000000  17348.000000   
2023-03-02 10:00:00  17346.699219  17374.199219  17344.449219  17354.349609   
2023-03-02 10:15:00  17354.500000  17367.849609  17345.699219  17362.650391   
2023-03-02 10:26:46  17367.349609  17367.349609  17367.349609  17367.349609   

                        Adj Close  Volume          5EMA  
Datetime                                                 
2023-03-02 09:30:00  17388.150391       0  17413.328547  
2023-03-02 09:45:00  17348.000000       0  17391.552365  
2023-03-02 10:00:00  17354.349609       0  17379.151446  
2023-03-02 10:15:00  17362.650391       0  17373.651095  
2023-03-02 10:26:46  17367.34

[*********************100%***********************]  1 of 1 completed
                             Open          High           Low         Close  \
Datetime                                                                      
2023-03-02 09:30:00  17375.750000  17393.550781  17364.150391  17388.150391   
2023-03-02 09:45:00  17388.750000  17393.449219  17348.000000  17348.000000   
2023-03-02 10:00:00  17346.699219  17374.199219  17344.449219  17354.349609   
2023-03-02 10:15:00  17354.500000  17369.550781  17345.699219  17369.550781   
2023-03-02 10:27:24  17371.599609  17371.599609  17371.599609  17371.599609   

                        Adj Close  Volume          5EMA  
Datetime                                                 
2023-03-02 09:30:00  17388.150391       0  17413.328547  
2023-03-02 09:45:00  17348.000000       0  17391.552365  
2023-03-02 10:00:00  17354.349609       0  17379.151446  
2023-03-02 10:15:00  17369.550781       0  17375.951225  
2023-03-02 10:27:24  17371.59

[*********************100%***********************]  1 of 1 completed
                             Open          High           Low         Close  \
Datetime                                                                      
2023-03-02 09:30:00  17375.750000  17393.550781  17364.150391  17388.150391   
2023-03-02 09:45:00  17388.750000  17393.449219  17348.000000  17348.000000   
2023-03-02 10:00:00  17346.699219  17374.199219  17344.449219  17354.349609   
2023-03-02 10:15:00  17354.500000  17373.750000  17345.699219  17373.050781   
2023-03-02 10:28:06  17372.449219  17372.449219  17372.449219  17372.449219   

                        Adj Close  Volume          5EMA  
Datetime                                                 
2023-03-02 09:30:00  17388.150391       0  17413.328547  
2023-03-02 09:45:00  17348.000000       0  17391.552365  
2023-03-02 10:00:00  17354.349609       0  17379.151446  
2023-03-02 10:15:00  17373.050781       0  17377.117891  
2023-03-02 10:28:06  17372.44

[*********************100%***********************]  1 of 1 completed
                             Open          High           Low         Close  \
Datetime                                                                      
2023-03-02 09:30:00  17375.750000  17393.550781  17364.150391  17388.150391   
2023-03-02 09:45:00  17388.750000  17393.449219  17348.000000  17348.000000   
2023-03-02 10:00:00  17346.699219  17374.199219  17344.449219  17354.349609   
2023-03-02 10:15:00  17354.500000  17373.750000  17345.699219  17373.050781   
2023-03-02 10:28:48  17371.150391  17371.150391  17371.150391  17371.150391   

                        Adj Close  Volume          5EMA  
Datetime                                                 
2023-03-02 09:30:00  17388.150391       0  17413.328547  
2023-03-02 09:45:00  17348.000000       0  17391.552365  
2023-03-02 10:00:00  17354.349609       0  17379.151446  
2023-03-02 10:15:00  17373.050781       0  17377.117891  
2023-03-02 10:28:48  17371.15

[*********************100%***********************]  1 of 1 completed
                             Open          High           Low         Close  \
Datetime                                                                      
2023-03-02 09:30:00  17375.750000  17393.550781  17364.150391  17388.150391   
2023-03-02 09:45:00  17388.750000  17393.449219  17348.000000  17348.000000   
2023-03-02 10:00:00  17346.699219  17374.199219  17344.449219  17354.349609   
2023-03-02 10:15:00  17354.500000  17373.750000  17345.699219  17371.000000   
2023-03-02 10:29:30  17366.000000  17366.000000  17366.000000  17366.000000   

                        Adj Close  Volume          5EMA  
Datetime                                                 
2023-03-02 09:30:00  17388.150391       0  17413.328547  
2023-03-02 09:45:00  17348.000000       0  17391.552365  
2023-03-02 10:00:00  17354.349609       0  17379.151446  
2023-03-02 10:15:00  17371.000000       0  17376.434298  
2023-03-02 10:29:30  17366.00

KeyboardInterrupt: 